# Text Mining met Python - OVV Rapporten

|   |   |
|------|------|
|__Auteur__ | Marlon van den Hoek  |
| __E-mail__  | <m.vandenhoek@onderzoeksraad.nl>  |
| __Onderwerpen__ | text mining, cleansing, tokenizing, stemming, stopwords, TF-IDF, Jaccard similarity, KMeans, hierarchical document clustering, LDA, sentiment analysis |
| __Afhankelijkheden__ | `nltk`, `textmining`, `pdfminer`, `re`, `multiprocessing`, `sklearn`, `pandas`, `gensim` |
| __Versie__ | `0.6` |

_Opmerking 1_: Om dit Jupyter Notebook correct te kunnen runnen moeten eerst de juiste pakketten (zie afhankelijkheden) geinstalleerd worden. Dit kan bijvoorbeeld met `pip`. Zie hiervoor de eerste cell hier onder.

_Opmerking 2_: Dit notebook is standaard beperkt tot 30 documenten. Om alle documenten te analyseren, verwijder de limiet van 30 bij het laden: verander `arange` bij de for loop, `job` bij de multicore loop.

In [ ]:
# To install the dependencies, run the following code -- run only once!!
! pip install nltk
! pip install textmining
! pip install pdfminer
! pip install multiprocessing
! pip install sklearn
! pip install gensim

## Introductie

In dit Jupyter notebook gaan we aan de slag met text mining op de (openbare) rapporten van de Onderzoeksraad. Met een ander script (scraper) zijn meer dan 600 rapporten van de OVV website gedownload. Daarin is rekening gehouden met andere documenten zoals errata, bijlagen, aanbevelingen; deze documenten zijn voor het grootste gedeelte uit de collectie gefilterd. Deze documenten zijn opgeslagen in `./Rapporten/`.

Omdat de meeste rapporten van de OVV zijn voorzien van een beveiliging tegen het kopieren van tekst, zijn alle rapporten eerst gedecrypt met een extern programma. De gedecrypte bestanden zijn te vinden in de map `./Rapporten decrypted/`. De onderliggende submappen geven de taal van het document aan: Nederlands `nl` of Engels `en`.

Voor het minen maken we gebruik van de natural language toolkit ([NLTK](http://www.nltk.org/)), het textmining pakket ([textmining1.0](https://pypi.python.org/pypi/textmining/1.0)) en het boek _Natural Language Processing with Python_<sup id="a1">[1](#f1)</sup>. Daarnaast maken we ook gebruik van pakketten voor lineaire algebra en matrix operaties, e.a. Deze pakketten importeren we eerst.

<b id="f1">1</b> Bird, S., Klein, E., & Loper, E. (2009). Natural language processing with Python: analyzing text with the natural language toolkit. "O'Reilly Media, Inc.". [↩](#a1)

In [1]:
# import dependencies
import numpy as np
import nltk, textmining

## Text laden

Als we gebruik willen maken van onze eigen corpus, database met text, dan moeten we de rapporten die gedownload zijn in pdf eerst converteren naar plain text (txt). Hiervoor gebruiken we `pdfminer`. Metadata halen we uit de pdf bestanden met het pakket `pyPdf` omdat deze sneller is.

In de volgende code importeren we eerst alle pakketten die nodig zijn, daarna definieren we de bestandlocatie en creeeren we een functie om een bestand te kunnen lezen. In deze functie wordt elke pagina van het desbetreffende bestand geparsed en bij elkaar gevoegd.

In [2]:
# import packages
import sys
import os
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
from cStringIO import StringIO
from pyPdf import PdfFileReader # extract pdf metadata

# Define language
lang = 'en'

# Define location of reports
FilePath = './Rapporten decrypted/' + lang + '/'

# Get files in folder
FileNames = os.listdir(FilePath)

# Make function to convert pdf to text
def pdfparser(FilePath,FileName): 
    fp = file(FilePath+FileName, 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    # Process each page contained in the document.
    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        data =  retstr.getvalue()
    
    # return text
    return data


Nu de functie om een bestand te lezen gedefineerd is kunnen we deze functie toe gaan passen op alle bestanden. We doen dit door over de bestanden heen te loopen.

In de loop gebeurt het volgende: 
1. het bestand wordt gelezen
2. de volledige, onveranderde tekst wordt toegevoegd aan matrix `fulltextmat` om later te gebruiken. Sommige functies vragen een ander tekstformaat, daarom slaan we de tekst op verschillende manieren op.
3. met het Regexp `re` pakket splitten we de tekst
4. converteren de tekst naar lower case
5. uitfilteren van lege indices
6. uitfilteren van numerieke waarden
7. als de overgebleven body niet `None` is dan voegen we de tokens toe aan de `textbody` array (alle woorden achter elkaar), slaan de tokens op in een dict `textdict` en als laatste voegen we de tokens van een tekst ook nog toe aan de matrix `textmat`. 

_**Let op**_ om geparallelliseerd bestanden te lezen en converteren, sla de volgende stappen over en ga naar [hier](#parallel). Als je systeem voorzien is van meerdere cores kan dit significant sneller zijn!

In [ ]:
# use regexp package to split
import re

#textbody = np.array([])
#fulltextmat = []   # update: niet meer nodig! laten staan ivm fallback
textdict = {}
metadict = {}
#textmat = []
#for i in np.arange(np.shape(FileName)[0]): # use this line for all files
for i in np.arange(0,30): # temp only take first 30 files
    print "Processing ",i+1,'/',np.shape(FileNames)[0],': ', FileNames[i],'\n'
    
    try:
        data = pdfparser(FilePath,FileNames[i])
        
        # add to fulltextmat
        #fulltextmat.append(data)
        
        # get PDF metadata
        pdfi = PdfFileReader(open("".join([FilePath,FileNames[i]]), "rb"))
        pdfi = pdfi.getDocumentInfo()
        metadict[FileNames[i]] = pdfi

        # split text
        data = np.array(re.split('\W+',data),dtype=str) # tokenize text
        
        # convert all data to lower case
        data = np.array([w.lower() for w in data])

        # filter out empty cells
        ind = data != ''
        data = data[ind]

        # filter out non-alpha strings
        ind = np.char.isalpha(data)
        data = data[ind]

        if data.size is not 0:
            # filter out strings with length = 1
            strlen = np.vectorize(len)
            ind = strlen(data) != 1
            data = data[ind]

            # add to textbody array
            #textbody = np.append(textbody,data)
        
            # also add data to dict and matrix for later use
            textdict[FileNames[i]] = list(data)
            #textmat.append(list(data))

    except ValueError:
        print "Skipping ", FileNames[i]

### Geparallelliseerd bestanden lezen en converteren
 <a name="parallel"></a>
 
Om een groot gedeelte van de `for` loops sneller te maken, kunnen we de processen parallelliseren met `multiprocessing`. Dit pakket moeten we eerst importeren en dan definieren wat we ermee willen doen.

Om het uitlezen van bestanden te kunnen faciliteren moeten we ook eerst een functie definieren om een bestand te lezen en cleanen, deze functie noemen we _processandtokenize_. Deze functie wordt parallel uitgevoerd en de output is een dict met daarin al onze bestandnamen en bijbehorende tokens.

_Let op_: het runnen van onderstaande code is erg CPU intensief en kan ervoor zorgen dat je systeem langzaam wordt gedurende het processen van de data.

In [3]:
import multiprocessing

# set number of processing cores to <max_cores> - 1
num_cores = multiprocessing.cpu_count() - 1

In [4]:
import re

def processandtokenize(FileName, dictname, metadict): 
    
    # print progress, var FileNames is taken from shared memory, not required as function input
    print "Processing: ",int(np.arange(0,len(FileNames))[np.array(FileNames) == FileName])+1,'/',len(FileNames), FileName
    #print "Processing ", FileName
    
    try:
        FilePath = './Rapporten decrypted/'+lang+'/'
        data = pdfparser(FilePath,FileName)
        
        # get PDF metadata
        pdfi = PdfFileReader(open("".join([FilePath,FileName]), "rb"))
        pdfi = pdfi.getDocumentInfo()
        metadict[FileName] = pdfi

        # split text
        data = np.array(re.split('\W+',data),dtype=str) # tokenize text
        
        # convert all data to lower case
        data = np.array([w.lower() for w in data])

        ## filter out empty cells
        ind = data != ''
        data = data[ind]

        ## filter out non-alpha strings
        ind = np.char.isalpha(data)
        data = data[ind]

        if data.size is not 0:
            # filter out strings with length = 1
            strlen = np.vectorize(len)
            ind = strlen(data) != 1
            data = data[ind]
        
        # add data to dict
        dictname[FileName] = list(data)

    except ValueError:
        print "Skipping ", FileName
        

# initialize multi-core process
manager = multiprocessing.Manager()
textdict = manager.dict()
metadict = manager.dict()

# define job
job = [multiprocessing.Process(target=processandtokenize, args=(i, textdict, metadict)) for i in FileNames[0:30]] # first 30 files
#job = [multiprocessing.Process(target=processandtokenize, args=(i, textdict, metadict)) for i in FileNames] # for all files


# start all jobs in the job list
_ = [p.start() for p in job]
_ = [p.join() for p in job]

# convert the multiprocess textdict opject to normal dict
textdict = dict(textdict)
metadict = dict(metadict)

Processing:  1 / 221 1f850875f33erapport-treinontsporing-hilversum-en-interactief.pdf
Processing:  2 / 221 2bf2ed16223ab-rapport-annelies-ilena-en.pdf
Processing:  3 / 221 2e46783a2333rapport-direct-teruggekeerd-na-rook-in-cockpit-bw-en-web.pdf
Processing:  4 / 221 4a5183767e76b-rapport-runway-incursion-en.pdf
Processing:  5 / 221 4d4b4bc7f9ddb-rapport-verlies-van-controle-tijdens-doorstart-en.pdf
Processing:  6 / 221 4d4057d68fe4ovv-semester-scheepvaartongevallen-5-en.pdf
Processing:  7 / 221 5c4ffd1921dbb-rapport-diamond-da-40d-en-interactief.pdf
Processing:  8 / 221 6a2c806849e0report-mh17-brochure-crash.pdf
Processing:  9 / 221 7df8df730b98publieksversie-azoresborg-web.pdf
Processing:  10 / 221 7f249d3143fepublieksuitgave-flinter-aland-en-web.pdf
Processing:  11 / 221 8a23d852303d20172585-b-rapport-atlantic-dawn-en-170823.pdf
Processing:  12 / 221 8ac71529b0892012037-rapport-overig-verkeer-in-nabijheid-en-def.pdf
Processing:  14 / 221 9fca1f1c7f3fb-rapport-aanvaring-in-ankergebied-

In [11]:
print 'Total number of tokens: ',np.shape(sum(list(textdict.values()),[]))[0]


Total number of tokens:  195856


## Text stemming

Nu alle woorden uit de text gehaald zijn en in een `numpy` array staan kunnen we de woorden gaan _stemmen_. We gebruiken hiervoor de Porter stemmer voor het Engels en de Snowball stemmer in de NLTK toolkit voor het Nederlands.

Let op: de huidige implementatie van de Snowball stemmer is niet erg doeltreffend voor de Nederlandse taal.

In [ ]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer

print "Supported Languages Snowball: \n"
print(" ".join(SnowballStemmer.languages))
print "\n"

print "Supported Languages Porter: \n"
print "english\n"

print "Processing ", lang, "stopwords\n"

# create a new instance
if lang == 'nl':
    stemmer = SnowballStemmer("dutch")
elif lang == 'en':
    stemmer = PorterStemmer()

for i in np.arange(0,np.shape(textdict.keys())[0]):
    text_stemmed = []
    print i+1,'/',np.shape(textdict.keys())[0]
    for j in np.arange(0,np.shape(textdict.values()[i])[0]):
        text_stemmed.append(stemmer.stem(textdict.values()[i][j]))
    textdict[textdict.keys()[i]] = text_stemmed


## Stopwords verwijderen

Om onze teksten te processen moeten we nog een cleansing stap uitvoeren: het verwijderen van de stopwoorden. Hiervoor gebruiken we de standaard NL/EN library in de NLTK toolkit.

In [ ]:
from nltk.corpus import stopwords
from collections import Counter

# filter NL en EN stopwoorden uit de tekst
#if lang == 'en':
#    data_filtered = np.array([w for w in data_filtered if not w in stopwords.words('english')])
#elif lang == 'nl':
#    data_filtered = np.array([w for w in textbody if not w in stopwords.words('dutch')])

for i in np.arange(0,np.shape(textdict.keys())[0]):
    print i+1,'/',np.shape(textdict.keys())[0]
    text_nostopw = [w for w in textdict.values()[i] if not w in stopwords.words('english') and not w in stopwords.words('dutch')]
      #  text_nostopw = [w for w in textdict.values()[i] if not w in stopwords.words('dutch') ]
    textdict[textdict.keys()[i]] = text_nostopw

# trick to flatten matrix: sum(list(textdict.values()),[])
count = Counter(sum(list(textdict.values()),[]))

print count.most_common(100)



## Jaccard Similarity

Om verschillende documenten met elkaar te vergelijken maken we gebruik van de _Jaccard Similarity_: $J\left(A,B\right) = \frac{|A\bigcap B|}{|A\bigcup B|}$. Dit is de lengte van de intersectie van de set tokens van tekst A en tekst B gedeeld door de union van beide sets.

In [ ]:
# define Jaccard similarity
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return float(len(intersection))/len(union)

# calculate the Jaccard similarity of text 1 and text 2
print jaccard_similarity(textdict.values()[0],textdict.values()[1])


De uitkomst van de Jaccard Similarity geeft de numerieke overeenkomst van beide teksten. In het geval veel stopwoorden in de tekst te vinden zijn is de kans groot dat hierdoor de relevantie omhoog geholpen wordt. Het is daarom belangrijk, alvorens de Jaccard Similarity uit te rekenen, om de stopwords te verwijderen. 

Om de intersectie van beide teksten te bekijken, gebruik het volgende commando:

In [ ]:
# to check the intersection of both texts:
set(textdict.values()[0]).intersection(set(textdict.values()[1]))

## TF-IDF - _term frequency-inverse document frequency_

Om het belang van verschillende woorden in een corpus numeriek uit te drukken berekenen we de TF-IDF. Dit doen we door gebruik te maken van de functie `feature_extraction` in de `sklearn` toolbox (onderdeel van `scikit`). 

De TF-IDF measure reflecteert hoe belangrijk een woord voor een document is door in eerste instantie te kijken hoe vaak een woord voor komt in een tekst. In het meest simpele geval gebruiken we hiervoor de woordfrequentie $f_{t,d}$. Ons algorimte maakt gebruik van de geaugmenteerde woordfrequentie $tf$:

$\text{tf}(t,d) = 0.5 + \frac{0.5 ~\times ~f(t,d)}{\text{max}\left\{ f(w,d):~w~\in ~d\right\}}$

waarin de noemer de maximale woordfrequentie aangeeft.

Omdat sommige woorden vaak in tekst aanwezig zijn maar eigenlijk weinig betekenis hebben maken we gebruik van de inverse document frequentie om de uitkomst van de woord frequentie te reguleren. De IDF is gedefinieerd als:

$ \text{idf}(t) = \log \frac{n_d}{df\left( d,t \right)} + 1$

hierin is $n_d$ het totaal aantal documenten en $df\left(t,d\right)$ het aantal documenten dat term $t$ bevat.

De TF-IDF krijgen we dan door:

$ \text{tf-idf}(t,d) = \text{tf}(t,d) \times \text{idf}(t)$

De TF-IDF wordt berekend voor alle unieke termen $t$ in alle unieke documenten $d$. De resulterende matrix heeft dus de dimensie $\left( t,d \right)^{\intercal}$ waarin element ($t_i,d_j)^{\intercal}$ de numerieke relevantie geeft van term $i$ in document $j$.

__**Limitatie bag of words**__
Omdat een simpel bag of words model geen rekening houdt met verkeerd gespelde woorden/typo's kunnen deze woorden als verschillend worden gezien door het algoritme. Om dit te voorkomen kunnen we gebruik maken van een collectie bigrams (n=2) of ngrams (n>2). Met een analyzer (in dit geval `char_wb`) en een gedefinieerde `ngram_range` kunnen we woorden dan herkennen aan $n$ features. Zo kunnen we voorkomen dat simpele fouten invloed hebben op het model.

**Let op**: hoe meer ngrams features, hoe meer geheugen nodig is!

Omdat `Tfidfvectorizer` alleen de volledige tekst als string accepteert moeten we eerst de tokens in `textdict` manipuleren -> we voegen de tokens samen tot een string en splitten die dan weer op de spatie tussen woorden. Dit is een workaround en genereert een kleine overhead.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# manipulate variable textdict to allow feeding into TfidfVectorizer function
alltext = [' '.join(text) for text in textdict.values()]

# define tokenizer
# dummy function: we split concatenated tokens at space
tfidf_tokenizer = lambda doc: [word for word in doc.split(' ')]

# define numer of ngrams
ngrams = (2,2)

# define vectorizer -- using ngrams
#sklearn_tfidf = TfidfVectorizer(analyzer='char_wb',ngram_range=ngrams,norm='l2',min_df=0,use_idf=True,smooth_idf=False, sublinear_tf=True, tokenizer=tfidf_tokenizer)

# define vectorizer -- unigram
sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0,use_idf=True,smooth_idf=False, sublinear_tf=True, tokenizer=tfidf_tokenizer)

tfidf_matrix = sklearn_tfidf.fit_transform(alltext)

print tfidf_matrix.toarray()[0].tolist()


In [ ]:
np.shape(tfidf_matrix)

(zie ook http://billchambers.me/tutorials/2014/12/21/tf-idf-explained-in-python.html)

In [ ]:
# function to count occurrence of every token
#def term_frequency(term, tokenized_document):
#    return tokenized_document.count(term)
#
#def sublinear_term_frequency(term, tokenized_document):
#    return 1 + math.log(tokenized_document.count(term))
#
#def augmented_term_frequency(term, tokenized_document):
#    max_count = max([term_frequency(t, tokenized_document) for t in tokenized_document])
#    return (0.5 + ((0.5 * term_frequency(term, tokenized_document))/max_count))
#
#def inverse_document_frequencies(tokenized_documents):
#    idf_values = {}
#    all_tokens_set = set([item for sublist in tokenized_documents for item in sublist])
#    for tkn in all_tokens_set:
#        contains_token = map(lambda doc: tkn in doc, tokenized_documents)
#        idf_values[tkn] = 1 + math.log(len(tokenized_documents)/(sum(contains_token)))
#    return idf_values
#
#def tfidf(tokenized_documents):
#    idf = inverse_document_frequencies(tokenized_documents)
#    tfidf_documents = []
#    for document in tokenized_documents:
#        doc_tfidf = []
#        for term in idf.keys():
#            tf = sublinear_term_frequency(term, document)
#            doc_tfidf.append(tf * idf[term])
#        tfidf_documents.append(doc_tfidf)
#    return tfidf_documents


# OLD !!!!!!! DO NOT REMOVE !!!!!!!

# sklearn implementation

# define a tokenizer function
#from nltk.corpus import stopwords
#tokenize = lambda doc: [word for word in re.split('\W+', doc.lower()) if np.char.isalpha(word) 
#                         and len(word)>1 and word not in stopwords.words('dutch') and word not in
#                         stopwords.words('english')]
#
#from sklearn.feature_extraction.text import TfidfVectorizer
#sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0,use_idf=True,smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
#
#tfidf_matrix = sklearn_tfidf.fit_transform(fulltextmat)
#
#print tfidf_matrix.toarray()[0].tolist()

In [ ]:
terms = sklearn_tfidf.get_feature_names()
print terms

### KMeans clustering

Aan de hand van de TF-IDF matrix kunnen we de cosine similarity uitrekenen en de onderlinge (numerieke) afstand tussen verschillende text bestanden. Op basis van deze measure kan er dan met het KMeans algoritme een onderscheid gemaakt worden tussen verschillende clusters.

Het aantal clusters moeten we zelf definieren. Dit is onderhevig aan trial en error: het zoeken van het optimale aantal clusters is geen triviaal proces en wordt vaak aangepakt met een brute force aanpak. Dit komt dus neer op $n$ keer het algoritme zijn clusters laten bepalen voor $n$ verschillende clustergrootten en achteraf bepalen met welke clustergrootte het beste resultaat wordt bereikt.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

from sklearn.cluster import KMeans
num_clusters = 5
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

# print clusters
for i in np.arange(0,num_clusters):
    print "Cluster", i, ": \n"
    print np.array(textdict.keys())[np.array(km.labels_.tolist()) == i]
    print "\n\n"
    
clusters = km.labels_.tolist()
    

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
MDS()

# dimension reduction -> reduce multi-dimensional field to 2 dimensions for plotting
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
pos = mds.fit_transform(dist)
xs, ys = pos[:,0], pos[:,1]

# visualize clusters
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=textdict.keys())) 
groups = df.groupby('label')

fig, ax = plt.subplots(figsize=(20, 10)) # set size
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

cluster_names = ['Cluster '+str(w) for w in np.arange(np.min(clusters),np.max(clusters)+1,1)]
cluster_colors = ['red','green','blue','black','orange']*5

for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, 
            label=cluster_names[name], color=cluster_colors[name], 
            mec='none')
    ax.set_aspect('auto')
    ax.tick_params(\
        axis= 'x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
    ax.tick_params(\
        axis= 'y',         # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelleft='off')
    
ax.legend(numpoints=1)  #show legend with only 1 point

#add label in x,y position with the label as the film title
for i in range(len(df)):
    ax.text(df.ix[i]['x'], df.ix[i]['y'], df.ix[i]['title'], size=8)  

    
    
plt.show() #show the plot

#uncomment the below to save the plot if need be
#plt.savefig('clusters_small_noaxes.png', dpi=200)

### Hierarchical document clustering


In [ ]:
from scipy.cluster.hierarchy import ward, dendrogram
import matplotlib.pyplot as plt
%matplotlib inline

dist = 1 - cosine_similarity(tfidf_matrix)
linkage_matrix = ward(dist)

fig, ax = plt.subplots(figsize=(15, 15./60*len(textdict.keys())))

ax = dendrogram(linkage_matrix, orientation="right", labels=textdict.keys());

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout
plt.savefig('Rapport-dendogram_plot.pdf', dpi=80)

## Latent Dirichlet Allocation - Topic Modeling

Om meer van de verborgen structuur in de tekstdocumenten te vinden kunnen we gebruik maken van _Latend Dirichlet Allocation_. LDA is een propabilistisch topic model die aanneemt dat documenten een mix zijn van verschillende onderwerpen (topics) en dat elk woord in het document bijdraagt aan dit onderwerp (en zo komt het onderwerp tot stand).

Om LDA te gebruiken moeten we eerst een model trainen met de data (teksten) die we hebben.

Eerst importeren we de benodigde toolkits:

In [ ]:
import string
from gensim import corpora, models, similarities

We maken nu een dictionary van alle tokens in onze `textdict` variabele.

In [ ]:
# create a Gensim dictionary from the texts
dictionary = corpora.Dictionary(textdict.values())

# remove extremes
dictionary.filter_extremes(no_below=1, no_above=0.8)

# convert dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in textdict.values()]

In [ ]:
# train the LDA model - TAKES A LONG TIME! (30 docs = +-1 min)
%time lda = models.LdaModel(corpus, num_topics=5,id2word = dictionary,update_every=5,chunksize=100000,passes=50)

In [ ]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)

# show topics
topics_matrix

## Sentiment Analyse

Om het sentiment van een document in te kunnen schatten kunnen we gebruik maken van _Naive Bayes_. Voor dit model wordt tekst gerepresenteerd als _bag of words_. De nauwkeurigheid van deze methode is afhankelijk van de lexicologie we gebruiken. `NLTK` maakt gebruik van een eigen library.


### Overig

In [ ]:
import datetime

date = np.array([])

# convert metadict date field to date array
for i in np.arange(0,len(metadict)):
    year = metadict[metadict.keys()[i]]['/CreationDate'][2:6]
    month = metadict[metadict.keys()[i]]['/CreationDate'][6:8]
    day = metadict[metadict.keys()[i]]['/CreationDate'][8:10]

    # convert to python datetime object
    date = np.append(date,datetime.date(int(year), int(month), int(day)))
    

In [ ]:
# plot dates
import matplotlib.pyplot as plt
from matplotlib import dates as pltdates

fig = plt.figure()
ax = fig.gca()

# set plot properties
plt.title("Tijdlijn rapporten", fontsize=22)
fig.set_size_inches(20,20./60*len(textdict.keys()))
fig.gca().spines['left'].set_visible(False)
fig.gca().spines['right'].set_visible(False)
fig.gca().spines['top'].set_visible(False)
fig.gca().get_yaxis().set_ticks([]) # remove y ticks
fig.gca().axes.yaxis.set_ticklabels([]) # remove y labels
plt.grid(True,which='major',axis='x')

# plot data
xplot = np.arange(0,np.shape(textdict.keys())[0])
yind = np.argsort(date) # sort date for plotting
plt.scatter(date[yind], xplot, c=(75./256,151./256,182./256),s=100) 
for i in np.arange(0,np.shape(textdict.keys())[0]):
    plt.text(date[yind[i]]+datetime.timedelta(60), i, metadict.keys()[yind[i]], fontsize=8)
    
plt.xticks(np.arange(datetime.date(min(date).year,1,1),datetime.date(max(date).year+1,1,1),365),fontsize=16,rotation=70)
    
# make sure text fits in axis
plt.xlim(fig.gca().get_xlim()[0],fig.gca().get_xlim()[1]*1.003)

# show figure and save figure to working directory
plt.savefig('Rapport-datum_plot.pdf', dpi=80, orientation='landscape')
plt.show()


# IDEE

Tekst met keywords in pdf markeren, zie voorbeelden om locatie van het woord te extracten en de tekst in het PDF bestand te highlighten:

- https://stackoverflow.com/questions/7605577/read-highlight-save-pdf-programatically
- https://stackoverflow.com/questions/22898145/how-to-extract-text-and-text-coordinates-from-a-pdf-file/
